<a href="https://colab.research.google.com/github/maylovesart/AI-for-Media-Exam/blob/main/UK_Black_Icons_GANs_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Set TensorFlow to use GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Load and Preprocess Images
TARGET_SIZE = (256, 256)

def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, TARGET_SIZE)
    img = img / 127.5 - 1  # Normalize to [-1, 1]
    return img

def load_and_preprocess_images(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(directory, filename)
            img = preprocess_image(img_path)
            images.append(img)
    return tf.stack(images)

# Load images from the specified directories
content_images = load_and_preprocess_images("/content/Diane Abbott")
style_images = load_and_preprocess_images("/content/Richard Stone")

print(f"Loaded {len(content_images)} content images and {len(style_images)} style images.")

# Data Augmentation
def augment_image(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image

# Spectral Normalization
class SpectralNormalization(tf.keras.constraints.Constraint):
    def __init__(self):
        self.iteration = 0

    def __call__(self, w):
        self.iteration += 1
        if self.iteration % 50 == 0:
            return tf.nn.l2_normalize(w, axis=None)
        return w

# GAN Model Definition
def make_generator_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(16 * 16 * 256, use_bias=False, input_shape=(200,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Reshape((16, 16, 256)),
        tf.keras.layers.Conv2DTranspose(128, 5, strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2DTranspose(64, 5, strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2DTranspose(32, 5, strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2DTranspose(3, 5, strides=2, padding='same', activation='tanh')
    ])
    return model

def make_discriminator_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, 5, strides=2, padding='same', input_shape=[256, 256, 3]),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(128, 5, strides=2, padding='same'),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(256, 5, strides=2, padding='same'),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)
    ])
    return model

# Loss Functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output) * 0.9, real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Learning Rate Schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.96
)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5)

# Training Step
@tf.function
def train_step(images):
    batch_size = tf.shape(images)[0]
    noise = tf.random.normal([batch_size, 200])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Generate and Save Images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, :] * 0.5 + 0.5)  # Denormalize
        plt.axis('off')

    plt.savefig(f'/content/generated_image_epoch_{epoch}.png')
    plt.close()

# Training Loop
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        # Print updates every 50 epochs
        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Time: {time.time()-start:.2f} sec")
            print(f"Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}")

        # Generate and save images every 100 epochs
        if (epoch + 1) % 100 == 0:
            generate_and_save_images(generator, epoch + 1, seed)

    # Generate a final set of images
    generate_and_save_images(generator, epochs, seed)

# Prepare the dataset
BATCH_SIZE = 32
EPOCHS = 1000

# Combine content and style images
combined_images = tf.concat([content_images, style_images], axis=0)
dataset = tf.data.Dataset.from_tensor_slices(combined_images)
dataset = dataset.map(augment_image)  # Apply data augmentation
dataset = dataset.shuffle(1000).batch(BATCH_SIZE, drop_remainder=True)

# Initialize models
generator = make_generator_model()
discriminator = make_discriminator_model()

# Apply spectral normalization to the discriminator
for layer in discriminator.layers:
    if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense)):
        layer.kernel_constraint = SpectralNormalization()

# Create a seed for image generation
seed = tf.random.normal([16, 200])

# Start training
print("Starting training for 1000 epochs...")
train(dataset, EPOCHS)
print("Training complete.")

# Ethical considerations and LLM disclaimer
print("\nEthical Considerations:")
print("This project involves generating images based on real individuals.")
print("It's important to consider the ethical implications of creating and using such images.")
print("The generated images should not be used for misrepresentation or without proper context.")

print("\nLLM Disclaimer:")
print("Large Language Models were used for code structuring and debugging assistance.")
print("All implementation decisions and parameter tuning were made by the human developer.")

Num GPUs Available:  1
Loaded 12 content images and 20 style images.
Starting training for 1000 epochs...
Epoch 50/1000, Time: 0.46 sec
Generator Loss: 3.3979, Discriminator Loss: 0.4020
Epoch 100/1000, Time: 0.43 sec
Generator Loss: 3.4230, Discriminator Loss: 0.4617
Epoch 150/1000, Time: 0.42 sec
Generator Loss: 4.5336, Discriminator Loss: 0.3754
Epoch 200/1000, Time: 0.43 sec
Generator Loss: 2.3865, Discriminator Loss: 0.5227
Epoch 250/1000, Time: 0.44 sec
Generator Loss: 4.0772, Discriminator Loss: 0.4585
Epoch 300/1000, Time: 0.43 sec
Generator Loss: 2.2390, Discriminator Loss: 0.5673
Epoch 350/1000, Time: 0.43 sec
Generator Loss: 2.6776, Discriminator Loss: 0.5783
Epoch 400/1000, Time: 0.43 sec
Generator Loss: 1.6828, Discriminator Loss: 0.6740
Epoch 450/1000, Time: 0.43 sec
Generator Loss: 2.1872, Discriminator Loss: 0.7476
Epoch 500/1000, Time: 0.43 sec
Generator Loss: 1.0608, Discriminator Loss: 0.8940
Epoch 550/1000, Time: 0.43 sec
Generator Loss: 1.7818, Discriminator Loss: 